In [2]:
import tensorflow as tf
from tensorflow.keras import layers, mixed_precision
import matplotlib.pyplot as plt
import os

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4050 Laptop GPU, compute capability 8.9


In [3]:
def cnn(input_shape, num_classes):
    model = tf.keras.Sequential([
        layers.Conv2D(64, (3, 3), activation = "relu", padding = "same", input_shape = input_shape),
        # layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation = "relu", padding = "same"),
        # layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation = "relu", padding = "same"),
        # layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation = "relu", padding = "same"),
        # layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        # layers.Conv2D(256, (3, 3), activation = "relu", padding = "same"),
        # layers.BatchNormalization(),
        # layers.Conv2D(256, (3, 3), activation = "relu", padding = "same"),
        # layers.BatchNormalization(),
        # layers.MaxPooling2D((2, 2)),

        # layers.Conv2D(512, (3, 3), activation = "relu", padding = "same"),
        # layers.BatchNormalization(),
        # layers.Conv2D(512, (3, 3), activation = "relu", padding = "same"),
        # layers.BatchNormalization(),
        # layers.MaxPooling2D((2, 2)),

        # layers.Conv2D(1024, (3, 3), activation = "relu", padding = "same"),
        # layers.BatchNormalization(),
        # layers.Conv2D(1024, (3, 3), activation = "relu", padding = "same"),
        # layers.BatchNormalization(),
        # layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(64, activation = "relu"),
        # layers.BatchNormalization(),
        
        layers.Dense(32, activation = "relu"),
        # layers.BatchNormalization(),

        layers.Dense(num_classes, activation = "softmax")
    ])

    return model

In [6]:
dir = "Dataset/seg_train/"

batch_size = 16
img_h, img_w = 150, 150

train_dataset = tf.keras.utils.image_dataset_from_directory(dir, seed = 100, image_size = (img_h, img_w), batch_size = batch_size)

Found 14034 files belonging to 6 classes.


In [7]:
model = cnn(input_shape = (150, 150, 3), num_classes = 6)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 87616)             0         
                                                                 
 dense (Dense)               (None, 64)                5

In [12]:
model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

In [14]:
import gc

# Before training, clear memory
tf.keras.backend.clear_session()
gc.collect()

233

In [ ]:
# with tf.device("/GPU:0"):
history = model.fit(train_dataset, epochs = 10)

Epoch 1/10
423/878 [=============>................] - ETA: 7s - loss: 24.4225 - accuracy: 0.4178